# Stage LIESSE – Suppléments

Le but de ce notebook est de présenter quelques exemples supplémentaires d'utilisation de scikit-learn

## Chargement des librairies

In [ ]:
%pylab inline 

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
import pandas as pd

## Un exemple de surapprentissage

Reprenons la prédiction du sexe d'un manchot. Nous allons créer des variables supplémentaires, qui sont des combinaisons polynômiales de nos 3 variables, et faire sur-apprendre une régression logistique.

In [ ]:
# Charger les données
penguins = pd.read_csv("data/penguins.csv")
y = pd.Categorical(penguins["sex"]).astype('category').codes
X = penguins[["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]].to_numpy()

In [ ]:
# Séparer en jeu d'entraînement et jeu de test
from sklearn import model_selection
(X_train, X_test, y_train, y_test) = model_selection.train_test_split(X, y, test_size=0.3, random_state=25)

In [ ]:
# Centrer-réduire chaque variable avec StandardScaler()
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) # calcule les moyennes et écart-types de chaque variable sur le jeu d'entraînement
X_train_scaled = scaler.transform(X_train) # retire à chaque variable sa moyenne et son écart-type
X_test_scaled = scaler.transform(X_test) # retire à chaque variable sa moyenne et son écart-type (calculés sur le jeu d'entraînement)

In [ ]:
# Créer des combinaisons polynômiales de nos variables
polynomialisation = preprocessing.PolynomialFeatures(degree=5) # utilser des polynomes de degré 10
X_train_poly = polynomialisation.fit_transform(X_train_scaled) 
X_test_poly = polynomialisation.fit_transform(X_test_scaled)
X_train_poly.shape

In [ ]:
# Entrainer une régression logistique
from sklearn import linear_model
logreg = linear_model.LogisticRegression(penalty='none', solver='newton-cg', max_iter=1000)
logreg.fit(X_train_poly, y_train)

In [ ]:
# Matrice de confusion sur le jeu d'entrainement
from sklearn import metrics
metrics.plot_confusion_matrix(logreg, X_train_poly, y_train)

Le modèle ne fait aucune erreur sur le jeu d'entraînement.

In [ ]:
# Matrice de confusion sur le jeu de test
metrics.plot_confusion_matrix(logreg, X_test_poly, y_test)

Le modèle fait des erreurs sur le jeu de test.

## Un exemple de classification multiclasse

Nous allons maintenant essayer de prédire l'espèce d'un manchot à partir de la longueur et la hauteur de son bec.

In [ ]:
# Charger les données
penguins = pd.read_csv("data/penguins.csv")

In [ ]:
# Récupérer les noms d'espèces de manchots
np.unique(penguins["species"])

In [ ]:
# Visualisation de la hauteur versus longueur du bec, par espèce
for penguin_species in np.unique(penguins["species"]):
    plt.scatter(penguins.loc[penguins["species"] == penguin_species]["bill_length_mm"], 
                penguins.loc[penguins["species"] == penguin_species]["bill_depth_mm"], 
                label=penguin_species)
plt.xlabel("Longueur du bec (mm)")
plt.ylabel("Hauteur du bec (mm)")
plt.legend()

__Commentaire :__  Les données semblent assez bien séparées !

In [ ]:
# Extraire la matrice X et le vecteur y
X = penguins[["bill_length_mm", "bill_depth_mm"]].to_numpy()
y = pd.Categorical(penguins["species"]).astype('category').codes

In [ ]:
# Séparer en train/test
from sklearn import model_selection
(X_train, X_test, y_train, y_test) = model_selection.train_test_split(X, y, test_size=0.3, random_state=25, 
                                                                      stratify=y)

In [ ]:
# Instancier un obket de la classe KNeighborsClassifiers 
from sklearn import neighbors
knnclass = neighbors.KNeighborsClassifier(n_neighbors=15) # on utilise ici l'heuristique k=racine carrée(n)

In [ ]:
# Entrainer le classifieur
knnclass.fit(X_train, y_train)

In [ ]:
# Utiliser le classifieur pour prédire sur les données de test
y_test_pred = knnclass.predict(X_test)

In [ ]:
# Matrice de confusion  
from sklearn import metrics
metrics.plot_confusion_matrix(knnclass, X_test, y_test)

__Pour aller plus loin :__ En reprenant le code du notebook 3, optimisez la valeur du nombre de voisins.

## Un exemple de traitement des chiffres manuscrits

On va utiliser le jeu de données `digits` de `scikit-learn`. [Documentation](https://scikit-learn.org/stable/datasets/toy_dataset.html#digits-dataset)

In [ ]:
# Charger les données
from sklearn import datasets
digits = datasets.load_digits()

`digits.images` contient les observations sous formes d'images 8 pixels x 8 pixels. Chaque pixel est représenté par un nombre entre 0 et 255 correspondant à son niveau de gris. On peut les visualiser ainsi :

In [ ]:
first_image = digits.images[0] # premiere image du jeu de donnees
plt.imshow(first_image, 
           cmap=plt.cm.gray_r) # utiliser une color map "niveaux de gris"

`digits.data` contient les observations sous formes de vecteurs de taille 64 : on a mis bout à bout tous les pixels. On peut les visualiser aussi :

In [ ]:
first_image_vector = digits.data[0] # représentation vectorielle de la première image
fig = plt.figure(figsize=(15, 64))
plt.imshow(np.reshape(first_image_vector, (1, 64)), # imshow prend une matrice en entrée, pas un vecteur
           cmap=plt.cm.gray_r)

`digits.target` contient les étiquettes :

In [ ]:
np.unique(digits.target)

In [ ]:
print("Le jeu de données contient %d observations" % digits.target.shape[0])

In [ ]:
# Séparer les données en entrainement et test :
(X_train, X_test, y_train, y_test) = model_selection.train_test_split(digits.data, digits.target, test_size=0.3, random_state=25, 
                                                                      stratify=digits.target)

In [ ]:
# Entrainer un kNN
from sklearn import neighbors
knnclass = neighbors.KNeighborsClassifier(n_neighbors=7)
knnclass.fit(X_train, y_train)

In [ ]:
# Matrice de confusion sur le jeu de test
from sklearn import metrics
metrics.plot_confusion_matrix(knnclass, X_test, y_test)